# Question 1 of notebook

In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
from bs4 import BeautifulSoup

In [4]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(URL)

In [6]:
soup = BeautifulSoup(source.content,'html5lib')

In [15]:
# Creating a table

table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell={}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] =(((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')

        table_contents.append(cell)

#printing
df = pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [16]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [17]:
df.shape

(103, 3)

# Question 2 of Notebook

In [18]:
!pip install geocoder
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [19]:
CLIENT_ID = '4A4CSZDNWNJ1JOGR2JTJPB1FCBA3ZYXOYPMHFVXPJU3NWDEV' # Foursquare ID
CLIENT_SECRET = 'ZJD5DIBXO5LXKRVMR1E44LHUDUZU1NI5NNTAMTTQL3MMD1ZB' # Foursquare Secret
ACCESS_TOKEN = '32CJSKVT4SGJEDXCOYEJGDC50LVPWDX0JTZRG1EJWXJT2CBT' # FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4A4CSZDNWNJ1JOGR2JTJPB1FCBA3ZYXOYPMHFVXPJU3NWDEV
CLIENT_SECRET:ZJD5DIBXO5LXKRVMR1E44LHUDUZU1NI5NNTAMTTQL3MMD1ZB


In [20]:
postCodes = df['PostalCode']
postCodes.head()

0    M3A
1    M4A
2    M5A
3    M6A
4    M7A
Name: PostalCode, dtype: object

In [21]:
geolocator = Nominatim(user_agent = "foursquare_agent")


for codes in postCodes:
    location = geolocator.geocode(codes)
    while(location is None):
        location = geolocator.geocode(codes)
    latitude = location.latitude
    longitude = location.longitude
    print(latitude, longitude)

-12.19833375 -76.96230963246998
49.4842897 8.46700036809857
45.44058845 28.018025356938132
53.7941642 -1.7520058
44.42768855 26.17130784385282


KeyboardInterrupt: 

In [22]:
#Reading csv file's data into a pandas dataframe

geoData = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geoData = geoData.rename(columns={'Postal Code':'PostalCode'})
geoData.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
new_df = pd.merge(df,geoData,on='PostalCode')
new_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Question 3 of Notebook

In [24]:
toronto_map = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

for latitude, longitude in zip(new_df.Latitude, new_df.Longitude):
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
       # popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(toronto_map)

# display map
toronto_map